# Cleaning League of Legends data for analysis
---
---

## Import relevant modules

In [100]:
import os
import pandas as pd

## Read in the webscraped data

In [107]:
datadir = '/Users/jeremy_lehner/Documents/GitHub/metis_project2/data/raw/'
datafiles = next(os.walk(datadir))[2]

datafiles = [datadir + file for file in datafiles]

['/Users/jeremy_lehner/Documents/GitHub/metis_project2/data/raw/scraped_data_2019-07-16.csv',
 '/Users/jeremy_lehner/Documents/GitHub/metis_project2/data/raw/scraped_data_2019-07-17.csv']

In [112]:
raw_dfs = []
for file in datafiles:
    raw_dfs.append(pd.read_csv(file))

raw_df = pd.concat(raw_dfs, ignore_index = True)
raw_df.head(10)

,champion,release_date,last_patch,num_skins,win_rate,ban_rate,pick_rate,date_data
0,Aatrox,2013-06-13,9.12,5,47.71%,23.42%,11.20%,2019-07-16
1,Ahri,2011-12-14,8.20,10,52.00%,7.41%,11.13%,2019-07-16
2,Akali,2010-05-11,9.11,10,43.66%,8.51%,8.22%,2019-07-16
3,Alistar,2009-02-21,8.21,12,47.77%,1.15%,3.53%,2019-07-16
4,Amumu,2009-06-26,9.12,10,52.63%,2.67%,6.42%,2019-07-16
5,Anivia,2009-07-10,8.24,8,50.09%,1.32%,3.34%,2019-07-16
6,Annie,2009-02-21,8.17,11,50.24%,1.44%,3.36%,2019-07-16
7,Ashe,2009-02-21,9.12,10,51.36%,1.98%,15.75%,2019-07-16
8,Aurelion Sol,2016-03-24,9.20,2,50.58%,0.32%,0.88%,2019-07-16
9,Azir,2014-09-16,9.70,4,46.99%,0.81%,3.85%,2019-07-16


## Clean up the win/ban/pick rate columns
---

### Win rates

In [113]:
# Remove % symbol
win_rates = raw_df['win_rate'].str.replace('%', '')

# Convert to floats and convert percent to proportion
win_rates = win_rates.astype('float')/100
win_rates.head()

0    0.4771
1    0.5200
2    0.4366
3    0.4777
4    0.5263
Name: win_rate, dtype: float64

### Ban rates

In [114]:
# Remove % symbol
ban_rates = raw_df['ban_rate'].str.replace('%', '')

# Convert to floats and convert percent to proportion
ban_rates = ban_rates.astype('float')/100
ban_rates.head()

0    0.2342
1    0.0741
2    0.0851
3    0.0115
4    0.0267
Name: ban_rate, dtype: float64

### Pick rates

In [115]:
# Remove % symbol
pick_rates = raw_df['pick_rate'].str.replace('%', '')

# Convert to floats and convert percent to proportion
pick_rates = pick_rates.astype('float')/100
pick_rates.head()

0    0.1120
1    0.1113
2    0.0822
3    0.0353
4    0.0642
Name: pick_rate, dtype: float64

## Determine number of patches since last champion change

In [116]:
# Get current patch from date_date column (automate later)
current_patch = '9.13'

# Get oldest patch that a champion was last changed (automate later)
oldest_patch  = str(raw_df['last_patch'].astype(float).min())

# Construct list of patches from current_patch to oldest_patch
patches = '9.13 9.12 9.11 9.10 9.9 9.8 9.7 9.6 9.5 9.4 9.3 9.2 9.1 \
           8.24b 8.24 8.23 8.22 8.21 8.20 8.19 8.18 8.17 8.16 8.15 8.14 \
           8.13 8.12 8.11 8.10 8.9 8.8 8.7 8.6 8.5 8.4 8.3 8.2 8.1 \
           7.24b 7.24 7.23 7.22'.split()

In [117]:
# Set number of patches since last change to the index of patches for which last_patch matches
patches_since_change = pd.Series(patches.index(s) for s in raw_df.last_patch.astype(str))
patches_since_change.head(10)

0     1
1    36
2     2
3    17
4     1
5    14
6    21
7     1
8    11
9     6
dtype: int64

## Determine age of each champion in days

In [118]:
# Get date when data was scraped
scrape_date = raw_df.date_data[0]

# Convert to datetime object
scrape_date = pd.to_datetime(scrape_date)

In [119]:
# Convert release_date values to datetime objects
release_dates = raw_df['release_date']
release_dates = pd.to_datetime(release_dates)
release_dates.head(10)

0   2013-06-13
1   2011-12-14
2   2010-05-11
3   2009-02-21
4   2009-06-26
5   2009-07-10
6   2009-02-21
7   2009-02-21
8   2016-03-24
9   2014-09-16
Name: release_date, dtype: datetime64[ns]

In [120]:
# Calculate the age of each champion in days
champion_age = (scrape_date - release_dates).dt.days
champion_age.head(10)

0    2224
1    2771
2    3353
3    3797
4    3672
5    3658
6    3797
7    3797
8    1209
9    1764
Name: release_date, dtype: int64

## Construct clean data set to use for EDA and regression modeling

In [121]:
clean_cols = [raw_df['champion'], champion_age, patches_since_change,
              raw_df['num_skins'], win_rates, ban_rates, pick_rates]

colnames = 'champion days_old num_last_patch num_skins win_rate ban_rate pick_rate'.split()

clean_df = pd.concat(clean_cols, axis = 1)
clean_df.columns = colnames

clean_df.head(20)

,champion,days_old,num_last_patch,num_skins,win_rate,ban_rate,pick_rate
0,Aatrox,2224,1,5,0.4771,0.2342,0.1120
1,Ahri,2771,36,10,0.5200,0.0741,0.1113
2,Akali,3353,2,10,0.4366,0.0851,0.0822
3,Alistar,3797,17,12,0.4777,0.0115,0.0353
4,Amumu,3672,1,10,0.5263,0.0267,0.0642
5,Anivia,3658,14,8,0.5009,0.0132,0.0334
6,Annie,3797,21,11,0.5024,0.0144,0.0336
7,Ashe,3797,1,10,0.5136,0.0198,0.1575
8,Aurelion Sol,1209,11,2,0.5058,0.0032,0.0088
9,Azir,1764,6,4,0.4699,0.0081,0.0385


## Write clean data frame to csv file

In [122]:
filedir = '/Users/jeremy_lehner/Documents/GitHub/metis_project2/data/processed/clean_data.csv'
clean_df.to_csv (filedir, index = None, header = True)